In [ ]:
import os, sys
import rasterio as rio
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import fsspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
sys.path.append('/content/src')
from fluvius import normalized_diff

with open("/content/credentials") as f:
    env_vars = f.read().split("\n")

for var in env_vars:
    key, value = var.split(" = ")
    os.environ[key] = value

storage_options = {"account_name":os.environ["ACCOUNT_NAME"],
                   "account_key":os.environ["BLOB_KEY"]}

fs = fsspec.filesystem("az", **storage_options)

In [ ]:
src = "ana"
id = "29200000_00000013_2019-09-14"
composite = "cir"
with rio.Env(
        AZURE_STORAGE_ACCOUNT=os.environ["ACCOUNT_NAME"],
        AZURE_STORAGE_ACCESS_KEY=os.environ["BLOB_KEY"]
    ):
    with rio.open(f"az://modeling-data/chips/500m_cloudthr80_lulc_masking/{src}/{id}.tif") as ds:
        img = np.moveaxis(ds.read(), 0, -1)
    with rio.open(f"az://modeling-data/chips/500m_cloudthr80_lulc_masking/{src}/{id}_water.tif") as ds:
        water = ds.read(1)
    with fs.open(f"az://modeling-data/chips/qa/{composite}_500m_cloudthr80_lulc_masking/{src}_{id}.png", 'rb') as fn:
        rgb = Image.fromarray(np.array(Image.open(fn))[:, 0:round(water.shape[1]), :])


ndvi = normalized_diff(img[:, :, 4], img[:, :, 3])
ndvi_water = (water == 1) & (ndvi < 0.25)

In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(18,18))
fig.tight_layout(pad=3.0)
ax1.imshow(rgb, interpolation="nearest")
ax1.set_title(f"{composite.upper()} Image", fontsize=16)
ax2.imshow(water, cmap="hot", interpolation="nearest")
ax2.set_title("IO LULC Water Mask", fontsize=16)
plt3 = ax3.imshow(ndvi, cmap="hot", interpolation="nearest")
ax3.set_title("NDVI", fontsize=16)
divider = make_axes_locatable(ax3)
plt.colorbar(plt3, cax=divider.append_axes('right', size='5%', pad=0.05))
ax4.imshow(ndvi_water*1, cmap="gray", interpolation="nearest")
ax4.set_title("Combined Water/NDVI Mask", fontsize=16)

fig.show()